In [12]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression

# 1. Загрузка данных
data = pd.read_csv('Top 100 Greatest Hollywood Actors of All Time.csv')

# 2. Предварительная обработка данных
data = data.dropna()
data = data.drop(columns=['Date of Birth', 'Place of Birth'])  # Удаляем ненужные столбцы

# Удаление строк, где хотя бы одно значение равно 0
data = data[(data[['Oscars', 'Oscar Nominations', 'BAFTA Nominations', 'Golden Globe Nominations', 'BAFTA', 'Golden Globes']] != 0).all(axis=1)]

# Преобразуем столбцы в числовые (если они не числовые)
data['Oscars'] = pd.to_numeric(data['Oscars'], errors='coerce')
data['BAFTA'] = pd.to_numeric(data['BAFTA'], errors='coerce')
data['Golden Globes'] = pd.to_numeric(data['Golden Globes'], errors='coerce')

# 3. Определение признаков и целевой переменной
X = data[['Oscar Nominations', 'BAFTA Nominations', 'Golden Globe Nominations', 'BAFTA', 'Golden Globes']]
y = data['Oscars']  # Предсказываем количество Оскаров

# 4. Список моделей
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=10, random_state=42)  # Уменьшаем количество деревьев для тестирования
}

# 5. Кросс-валидация и результаты
results = {}

for model_name, model in models.items():
    # Кросс-валидация для оценки модели
    cv_mse_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    cv_mae_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')
    cv_r2_scores = cross_val_score(model, X, y, cv=5, scoring='r2')
    
    results[model_name] = {
        'Mean MSE (CV)': -np.mean(cv_mse_scores),
        'Mean MAE (CV)': -np.mean(cv_mae_scores),
        'Mean R² (CV)': np.mean(cv_r2_scores)
    }

# Выводим результаты кросс-валидации для каждой модели
for model_name, metrics in results.items():
    print(f'{model_name} - Mean MSE (CV): {metrics["Mean MSE (CV)"]}, Mean MAE (CV): {metrics["Mean MAE (CV)"]}, Mean R² (CV): {metrics["Mean R² (CV)"]}')

# 6. Обучение и оценка на тренировочных данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

best_model_name = None
best_average_metric = float('inf')  # для выбора наименьшего значения
best_model = None  # Инициализируем переменную для лучшей модели

# Обучаем модели и оцениваем их на тестовых данных
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f'Model: {model_name} - MSE: {mse}, MAE: {mae}, R²: {r2}')
    
    # Используем усредненные метрики на кросс-валидации для выбора лучшей модели
    average_metric = results[model_name]['Mean MSE (CV)'] + results[model_name]['Mean MAE (CV)']
    
    if average_metric < best_average_metric:
        best_average_metric = average_metric
        best_model_name = model_name
        best_model = model  # Сохраняем саму модель

# 7. Вывод лучшей модели
print(f'The best model is: {best_model_name}')

# 8. Сохранение лучшей модели
joblib.dump(best_model, 'best_actor_award_predictor.pkl')  # Сохраняем самую лучшую модель

print("Best model saved as best_actor_award_predictor.pkl")

print("Предсказанные значения:", y_pred)
print("Реальные значения:", y_test.values)
print(best_model.coef_)

Linear Regression - Mean MSE (CV): 0.3538005838775974, Mean MAE (CV): 0.44324703319645187, Mean R² (CV): -0.18289718016231465
Random Forest - Mean MSE (CV): 0.5208928571428572, Mean MAE (CV): 0.4853571428571429, Mean R² (CV): -0.6490688644688645
Model: Linear Regression - MSE: 0.2721298267241469, MAE: 0.4332630783151621, R²: 0.7097281848275767
Model: Random Forest - MSE: 0.8175, MAE: 0.7, R²: 0.128
The best model is: Linear Regression
Best model saved as best_actor_award_predictor.pkl
Предсказанные значения: [2.  1.4 1.9 1.4 1.6 1.  1.  1.5]
Реальные значения: [4 1 1 1 2 1 2 2]
[ 0.0593184  -0.02436145  0.01250937  0.23509037  0.05249075]
